<a href="https://colab.research.google.com/github/DJNahata/Style-GANS-Practice/blob/master/G_Synthesis_Network_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf

print("DJ1")

TensorFlow 1.x selected.
DJ1


In [0]:
!git clone https://github.com/NVlabs/stylegan.git

Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.


In [0]:
cd /content/stylegan/

/content/stylegan


In [0]:
import dnnlib
import dnnlib.tflib as tflib

def _blur2d(x, f=[1,2,1], normalize=True, flip=False, stride=1):
    assert x.shape.ndims == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(stride, int) and stride >= 1

    # Finalize filter kernel.
    f = np.array(f, dtype=np.float32)
    if f.ndim == 1:
        f = f[:, np.newaxis] * f[np.newaxis, :]
    assert f.ndim == 2
    if normalize:
        f /= np.sum(f)
    if flip:
        f = f[::-1, ::-1]
    f = f[:, :, np.newaxis, np.newaxis]
    f = np.tile(f, [1, 1, int(x.shape[1]), 1])

    # No-op => early exit.
    if f.shape == (1, 1) and f[0,0] == 1:
        return x

    # Convolve using depthwise_conv2d.
    orig_dtype = x.dtype
    x = tf.cast(x, tf.float32)  # tf.nn.depthwise_conv2d() doesn't support fp16
    f = tf.constant(f, dtype=x.dtype, name='filter')
    strides = [1, 1, stride, stride]
    x = tf.nn.depthwise_conv2d(x, f, strides=strides, padding='SAME', data_format='NCHW')
    x = tf.cast(x, orig_dtype)
    return x

def _upscale2d(x, factor=2, gain=1):
    assert x.shape.ndims == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >= 1

    # Apply gain.
    if gain != 1:
        x *= gain

    # No-op => early exit.
    if factor == 1:
        return x

    # Upscale using tf.tile().
    s = x.shape
    x = tf.reshape(x, [-1, s[1], s[2], 1, s[3], 1])
    x = tf.tile(x, [1, 1, 1, factor, 1, factor])
    x = tf.reshape(x, [-1, s[1], s[2] * factor, s[3] * factor])
    return x

def _downscale2d(x, factor=2, gain=1):
    assert x.shape.ndims == 4 and all(dim.value is not None for dim in x.shape[1:])
    assert isinstance(factor, int) and factor >= 1

    # 2x2, float32 => downscale using _blur2d().
    if factor == 2 and x.dtype == tf.float32:
        f = [np.sqrt(gain) / factor] * factor
        return _blur2d(x, f=f, normalize=False, stride=factor)

    # Apply gain.
    if gain != 1:
        x *= gain

    # No-op => early exit.
    if factor == 1:
        return x

    # Large factor => downscale using tf.nn.avg_pool().
    # NOTE: Requires tf_config['graph_options.place_pruned_graph']=True to work.
    ksize = [1, 1, factor, factor]
    return tf.nn.avg_pool(x, ksize=ksize, strides=ksize, padding='VALID', data_format='NCHW')

#----------------------------------------------------------------------------
# High-level ops for manipulating 4D activation tensors.
# The gradients of these are meant to be as efficient as possible.

def blur2d(x, f=[1,2,1], normalize=True):
    with tf.variable_scope('Blur2D'):
        @tf.custom_gradient
        def func(x):
            y = _blur2d(x, f, normalize)
            @tf.custom_gradient
            def grad(dy):
                dx = _blur2d(dy, f, normalize, flip=True)
                return dx, lambda ddx: _blur2d(ddx, f, normalize)
            return y, grad
        return func(x)

def upscale2d(x, factor=2):
    with tf.variable_scope('Upscale2D'):
        @tf.custom_gradient
        def func(x):
            y = _upscale2d(x, factor)
            @tf.custom_gradient
            def grad(dy):
                dx = _downscale2d(dy, factor, gain=factor**2)
                return dx, lambda ddx: _upscale2d(ddx, factor)
            return y, grad
        return func(x)

def downscale2d(x, factor=2):
    with tf.variable_scope('Downscale2D'):
        @tf.custom_gradient
        def func(x):
            y = _downscale2d(x, factor)
            @tf.custom_gradient
            def grad(dy):
                dx = _upscale2d(dy, factor, gain=1/factor**2)
                return dx, lambda ddx: _downscale2d(ddx, factor)
            return y, grad
        return func(x)

#----------------------------------------------------------------------------
# Get/create weight tensor for a convolutional or fully-connected layer.

def get_weight(shape, gain=np.sqrt(2), use_wscale=False, lrmul=1):
    fan_in = np.prod(shape[:-1]) # [kernel, kernel, fmaps_in, fmaps_out] or [in, out]
    he_std = gain / np.sqrt(fan_in) # He init

    # Equalized learning rate and custom learning rate multiplier.
    if use_wscale:
        init_std = 1.0 / lrmul
        runtime_coef = he_std * lrmul
    else:
        init_std = he_std / lrmul
        runtime_coef = lrmul

    # Create variable.
    init = tf.initializers.random_normal(0, init_std)
    return tf.get_variable('weight', shape=shape, initializer=init) * runtime_coef

#----------------------------------------------------------------------------
# Fully-connected layer.

def dense(x, fmaps, **kwargs):
    if len(x.shape) > 2:
        x = tf.reshape(x, [-1, np.prod([d.value for d in x.shape[1:]])])
    w = get_weight([x.shape[1].value, fmaps], **kwargs)
    w = tf.cast(w, x.dtype)
    return tf.matmul(x, w)

#----------------------------------------------------------------------------
# Convolutional layer.

def conv2d(x, fmaps, kernel, **kwargs):
    assert kernel >= 1 and kernel % 2 == 1
    w = get_weight([kernel, kernel, x.shape[1].value, fmaps], **kwargs)
    w = tf.cast(w, x.dtype)
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME', data_format='NCHW')

#----------------------------------------------------------------------------
# Fused convolution + scaling.
# Faster and uses less memory than performing the operations separately.

def upscale2d_conv2d(x, fmaps, kernel, fused_scale='auto', **kwargs):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) * 2 >= 128

    # Not fused => call the individual ops directly.
    if not fused_scale:
        return conv2d(upscale2d(x), fmaps, kernel, **kwargs)

    # Fused => perform both ops simultaneously using tf.nn.conv2d_transpose().
    w = get_weight([kernel, kernel, x.shape[1].value, fmaps], **kwargs)
    w = tf.transpose(w, [0, 1, 3, 2]) # [kernel, kernel, fmaps_out, fmaps_in]
    w = tf.pad(w, [[1,1], [1,1], [0,0], [0,0]], mode='CONSTANT')
    w = tf.add_n([w[1:, 1:], w[:-1, 1:], w[1:, :-1], w[:-1, :-1]])
    w = tf.cast(w, x.dtype)
    os = [tf.shape(x)[0], fmaps, x.shape[2] * 2, x.shape[3] * 2]
    return tf.nn.conv2d_transpose(x, w, os, strides=[1,1,2,2], padding='SAME', data_format='NCHW')

def conv2d_downscale2d(x, fmaps, kernel, fused_scale='auto', **kwargs):
    assert kernel >= 1 and kernel % 2 == 1
    assert fused_scale in [True, False, 'auto']
    if fused_scale == 'auto':
        fused_scale = min(x.shape[2:]) >= 128

    # Not fused => call the individual ops directly.
    if not fused_scale:
        return downscale2d(conv2d(x, fmaps, kernel, **kwargs))

    # Fused => perform both ops simultaneously using tf.nn.conv2d().
    w = get_weight([kernel, kernel, x.shape[1].value, fmaps], **kwargs)
    w = tf.pad(w, [[1,1], [1,1], [0,0], [0,0]], mode='CONSTANT')
    w = tf.add_n([w[1:, 1:], w[:-1, 1:], w[1:, :-1], w[:-1, :-1]]) * 0.25
    w = tf.cast(w, x.dtype)
    return tf.nn.conv2d(x, w, strides=[1,1,2,2], padding='SAME', data_format='NCHW')

#----------------------------------------------------------------------------
# Apply bias to the given activation tensor.

def apply_bias(x, lrmul=1):
    b = tf.get_variable('bias', shape=[x.shape[1]], initializer=tf.initializers.zeros()) * lrmul
    b = tf.cast(b, x.dtype)
    if len(x.shape) == 2:
        return x + b
    return x + tf.reshape(b, [1, -1, 1, 1])

#----------------------------------------------------------------------------
# Leaky ReLU activation. More efficient than tf.nn.leaky_relu() and supports FP16.

def leaky_relu(x, alpha=0.2):
    with tf.variable_scope('LeakyReLU'):
        alpha = tf.constant(alpha, dtype=x.dtype, name='alpha')
        @tf.custom_gradient
        def func(x):
            y = tf.maximum(x, x * alpha)
            @tf.custom_gradient
            def grad(dy):
                dx = tf.where(y >= 0, dy, dy * alpha)
                return dx, lambda ddx: tf.where(y >= 0, ddx, ddx * alpha)
            return y, grad
        return func(x)

#----------------------------------------------------------------------------
# Pixelwise feature vector normalization.

def pixel_norm(x, epsilon=1e-8):
    with tf.variable_scope('PixelNorm'):
        epsilon = tf.constant(epsilon, dtype=x.dtype, name='epsilon')
        return x * tf.rsqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)

#----------------------------------------------------------------------------
# Instance normalization.

def instance_norm(x, epsilon=1e-8):
    assert len(x.shape) == 4 # NCHW
    with tf.variable_scope('InstanceNorm'):
        orig_dtype = x.dtype
        x = tf.cast(x, tf.float32)
        x -= tf.reduce_mean(x, axis=[2,3], keepdims=True)
        epsilon = tf.constant(epsilon, dtype=x.dtype, name='epsilon')
        x *= tf.rsqrt(tf.reduce_mean(tf.square(x), axis=[2,3], keepdims=True) + epsilon)
        x = tf.cast(x, orig_dtype)
        return x

#----------------------------------------------------------------------------
# Style modulation.

def style_mod(x, dlatent, **kwargs):
    with tf.variable_scope('StyleMod'):
        style = apply_bias(dense(dlatent, fmaps=x.shape[1]*2, gain=1, **kwargs))
        style = tf.reshape(style, [-1, 2, x.shape[1]] + [1] * (len(x.shape) - 2))
        return x * (style[:,0] + 1) + style[:,1]

#----------------------------------------------------------------------------
# Noise input.

def apply_noise(x, noise_var=None, randomize_noise=True):
    assert len(x.shape) == 4 # NCHW
    with tf.variable_scope('Noise'):
        if noise_var is None or randomize_noise:
            noise = tf.random_normal([tf.shape(x)[0], 1, x.shape[2], x.shape[3]], dtype=x.dtype)
        else:
            noise = tf.cast(noise_var, x.dtype)
        weight = tf.get_variable('weight', shape=[x.shape[1].value], initializer=tf.initializers.zeros())
        return x + noise * tf.reshape(tf.cast(weight, x.dtype), [1, -1, 1, 1])

In [0]:
def minibatch_stddev_layer(x, group_size=4, num_new_features=1):
    with tf.variable_scope('MinibatchStddev'):
        group_size = tf.minimum(group_size, tf.shape(x)[0])     # Minibatch must be divisible by (or smaller than) group_size.
        s = x.shape                                             # [NCHW]  Input shape.
        y = tf.reshape(x, [group_size, -1, num_new_features, s[1]//num_new_features, s[2], s[3]])   # [GMncHW] Split minibatch into M groups of size G. Split channels into n channel groups c.
        y = tf.cast(y, tf.float32)                              # [GMncHW] Cast to FP32.
        y -= tf.reduce_mean(y, axis=0, keepdims=True)           # [GMncHW] Subtract mean over group.
        y = tf.reduce_mean(tf.square(y), axis=0)                # [MncHW]  Calc variance over group.
        y = tf.sqrt(y + 1e-8)                                   # [MncHW]  Calc stddev over group.
        y = tf.reduce_mean(y, axis=[2,3,4], keepdims=True)      # [Mn111]  Take average over fmaps and pixels.
        y = tf.reduce_mean(y, axis=[2])                         # [Mn11] Split channels into c channel groups
        y = tf.cast(y, x.dtype)                                 # [Mn11]  Cast back to original data type.
        y = tf.tile(y, [group_size, 1, s[2], s[3]])             # [NnHW]  Replicate over group and pixels.
        return tf.concat([x, y], axis=1)

In [0]:
def G_synthesis12(
    dlatents_in,                        # Input: Disentangled latents (W) [minibatch, num_layers, dlatent_size].
    dlatent_size        = 512,          # Disentangled latent (W) dimensionality.
    num_channels        = 3,            # Number of output color channels.
    resolution          = 1024,         # Output resolution.
    fmap_base           = 8192,         # Overall multiplier for the number of feature maps.
    fmap_decay          = 1.0,          # log2 feature map reduction when doubling the resolution.
    fmap_max            = 512,          # Maximum number of feature maps in any layer.
    use_styles          = True,         # Enable style inputs?
    const_input_layer   = True,         # First layer is a learned constant?
    use_noise           = True,         # Enable noise inputs?
    randomize_noise     = True,         # True = randomize noise inputs every time (non-deterministic), False = read noise inputs from variables.
    nonlinearity        = 'lrelu',      # Activation function: 'relu', 'lrelu'
    use_wscale          = True,         # Enable equalized learning rate?
    use_pixel_norm      = False,        # Enable pixelwise feature vector normalization?
    use_instance_norm   = True,         # Enable instance normalization?
    dtype               = 'float32',    # Data type to use for activations and outputs.
    fused_scale         = 'auto',       # True = fused convolution + scaling, False = separate ops, 'auto' = decide automatically.
    blur_filter         = [1,2,1],      # Low-pass filter to apply when resampling activations. None = no filtering.
    structure           = 'auto',       # 'fixed' = no progressive growing, 'linear' = human-readable, 'recursive' = efficient, 'auto' = select automatically.
    is_template_graph   = False,        # True = template graph constructed by the Network class, False = actual evaluation.
    force_clean_graph   = False,        # True = construct a clean graph that looks nice in TensorBoard, False = default behavior.
    **_kwargs):                         # Ignore unrecognized keyword args.

    resolution_log2 = int(np.log2(resolution))
    assert resolution == 2**resolution_log2 and resolution >= 4
    def nf(stage): return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)
    def blur(x): return blur2d(x, blur_filter) if blur_filter else x
    if is_template_graph: force_clean_graph = True
    if force_clean_graph: randomize_noise = False
    if structure == 'auto': structure = 'linear' if force_clean_graph else 'recursive'
    act, gain = {'relu': (tf.nn.relu, np.sqrt(2)), 'lrelu': (leaky_relu, np.sqrt(2))}[nonlinearity]
    num_layers = resolution_log2 * 2 - 2
    num_styles = num_layers if use_styles else 1
    images_out = None

    # Primary inputs.
    dlatents_in.set_shape([None, num_styles, dlatent_size])
    dlatents_in = tf.cast(dlatents_in, dtype)
    with tf.variable_scope('lod', reuse = tf.AUTO_REUSE):
        lod_in = tf.cast(tf.get_variable('lod', initializer=np.float32(0), trainable=False), dtype)

    # Noise inputs.
    noise_inputs = []
    if use_noise:
      with tf.variable_scope('Noise', reuse=tf.AUTO_REUSE):
        for layer_idx in range(num_layers):
            res = layer_idx // 2 + 2
            shape = [1, use_noise, 2**res, 2**res]
            noise_inputs.append(tf.get_variable('noise%d' % layer_idx, shape=shape, initializer=tf.initializers.random_normal(), trainable=False))
    
    a1 = []
    a11 = []
    a12 = []
    a2 = []
    a3 = []
    a4 = []
    a5 = []
    a6 = []
    a71 = []
    a72 = []
    a8 = []
    a9 = []
    a10 = []

    a1.append(dlatents_in)


    # Things to do at the end of each layer.
    def layer_epilogue(x, a2, a3, a4, a5, a6, layer_idx):
        if use_noise:
            x = apply_noise(x, noise_inputs[layer_idx], randomize_noise=randomize_noise)
            a2.append(x)
        x = apply_bias(x)
        a3.append(x)
        x = act(x)
        a4.append(x)
        if use_pixel_norm:
            x = pixel_norm(x)
        if use_instance_norm:
            x = instance_norm(x)
            a5.append(x)
        if use_styles:
            x = style_mod(x, dlatents_in[:, layer_idx], use_wscale=use_wscale)
            a6.append(x)
        return x

    # Early layers.
    with tf.variable_scope('4x4', reuse = tf.AUTO_REUSE):
        if const_input_layer:
            with tf.variable_scope('Const'):
                x = tf.get_variable('const', shape=[1, nf(1), 4, 4], initializer=tf.initializers.ones())
                a1.append(x)
                x = layer_epilogue(tf.tile(tf.cast(x, dtype), [tf.shape(dlatents_in)[0], 1, 1, 1]), a2, a3, a4, a5, a6, 0)
        else:
            with tf.variable_scope('Dense'):
                x = dense(dlatents_in[:, 0], fmaps=nf(1)*16, gain=gain/4, use_wscale=use_wscale) # tweak gain to match the official implementation of Progressing GAN
                x = layer_epilogue(tf.reshape(x, [-1, nf(1), 4, 4]), 0)
        with tf.variable_scope('Conv', reuse = tf.AUTO_REUSE):
            x = conv2d(x, fmaps=nf(1), kernel=3, gain=gain, use_wscale=use_wscale)
            a1.append(x)
            x = layer_epilogue(x, a2, a3, a4, a5, a6, 1)

    # Building blocks for remaining layers.
    def block(res, a1, a2, a3, a4, a5, a6, a11, a12, x): # res = 3..resolution_log2
        with tf.variable_scope('%dx%d' % (2**res, 2**res), reuse = tf.AUTO_REUSE):
            with tf.variable_scope('Conv0_up'):
                x = upscale2d_conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale, fused_scale=fused_scale)
                a11.append(x)
                x = blur(x)
                a12.append(x)
                x = layer_epilogue(x, a2, a3, a4, a5, a6, res*2-4)
            with tf.variable_scope('Conv1'):
                x = conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale)
                a1.append(x)
                x = layer_epilogue(x, a2, a3, a4, a5, a6, res*2-3)
            return x


    def torgb(res, a71, a72, x): # res = 2..resolution_log2
        lod = resolution_log2 - res
        with tf.variable_scope('ToRGB_lod%d' % lod, reuse = tf.AUTO_REUSE):
            x = conv2d(x, fmaps=num_channels, kernel=1, gain=1, use_wscale=use_wscale)
            a71.append(x)
            x = apply_bias(x)
            a72.append(x)
            return x

    # Fixed structure: simple and efficient, but does not support progressive growing.
    if structure == 'fixed':
        for res in range(3, resolution_log2 + 1):
            x = block(res, x)
        images_out = torgb(resolution_log2, x)

    # Linear structure: simple but inefficient.
    if structure == 'linear':
        images_out = torgb(2, a71, a72, x)
        for res in range(3, resolution_log2 + 1):
            lod = resolution_log2 - res
            x = block(res, a1, a2, a3, a4, a5, a6, a11, a12, x)
            img = torgb(res, a71, a72, x)
            a10.append(img)
            images_out = upscale2d(images_out)
            with tf.variable_scope('Grow_lod%d' % lod, reuse = tf.AUTO_REUSE):
                images_out = tflib.lerp_clip(img, images_out, lod_in - lod)
                a8.append(images_out)

    # Recursive structure: complex but efficient.
    if structure == 'recursive':
        def cset(cur_lambda, new_cond, new_lambda):
            return lambda: tf.cond(new_cond, new_lambda, cur_lambda)
        def grow(x, res, lod):
            y = block(res, x)
            img = lambda: upscale2d(torgb(res, y), 2**lod)
            img = cset(img, (lod_in > lod), lambda: upscale2d(tflib.lerp(torgb(res, y), upscale2d(torgb(res - 1, x)), lod_in - lod), 2**lod))
            if lod > 0: img = cset(img, (lod_in < lod), lambda: grow(y, res + 1, lod - 1))
            return img()
        images_out = grow(x, 3, resolution_log2 - 3)

    a9.append(a1)
    a9.append(a11)
    a9.append(a12)
    a9.append(a2)
    a9.append(a3)
    a9.append(a4)
    a9.append(a5)
    a9.append(a6)
    a9.append(a71)
    a9.append(a72)
    a9.append(a10)
    a9.append(a8)

    assert images_out.dtype == tf.as_dtype(dtype)
    return a9, tf.identity(images_out, name='images_out')

In [0]:
def G_synthesis(
    dlatents_in,                        # Input: Disentangled latents (W) [minibatch, num_layers, dlatent_size].
    dlatent_size        = 512,          # Disentangled latent (W) dimensionality.
    num_channels        = 3,            # Number of output color channels.
    resolution          = 1024,         # Output resolution.
    fmap_base           = 8192,         # Overall multiplier for the number of feature maps.
    fmap_decay          = 1.0,          # log2 feature map reduction when doubling the resolution.
    fmap_max            = 512,          # Maximum number of feature maps in any layer.
    use_styles          = True,         # Enable style inputs?
    const_input_layer   = True,         # First layer is a learned constant?
    use_noise           = True,         # Enable noise inputs?
    randomize_noise     = True,         # True = randomize noise inputs every time (non-deterministic), False = read noise inputs from variables.
    nonlinearity        = 'lrelu',      # Activation function: 'relu', 'lrelu'
    use_wscale          = True,         # Enable equalized learning rate?
    use_pixel_norm      = False,        # Enable pixelwise feature vector normalization?
    use_instance_norm   = True,         # Enable instance normalization?
    dtype               = 'float32',    # Data type to use for activations and outputs.
    fused_scale         = 'auto',       # True = fused convolution + scaling, False = separate ops, 'auto' = decide automatically.
    blur_filter         = [1,2,1],      # Low-pass filter to apply when resampling activations. None = no filtering.
    structure           = 'auto',       # 'fixed' = no progressive growing, 'linear' = human-readable, 'recursive' = efficient, 'auto' = select automatically.
    is_template_graph   = False,        # True = template graph constructed by the Network class, False = actual evaluation.
    force_clean_graph   = False,        # True = construct a clean graph that looks nice in TensorBoard, False = default behavior.
    **_kwargs):                         # Ignore unrecognized keyword args.

    resolution_log2 = int(np.log2(resolution))
    assert resolution == 2**resolution_log2 and resolution >= 4
    def nf(stage): return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)
    def blur(x): return blur2d(x, blur_filter) if blur_filter else x
    if is_template_graph: force_clean_graph = True
    if force_clean_graph: randomize_noise = False
    if structure == 'auto': structure = 'linear' if force_clean_graph else 'recursive'
    act, gain = {'relu': (tf.nn.relu, np.sqrt(2)), 'lrelu': (leaky_relu, np.sqrt(2))}[nonlinearity]
    num_layers = resolution_log2 * 2 - 2
    num_styles = num_layers if use_styles else 1
    images_out = None

    # Primary inputs.
    dlatents_in.set_shape([None, num_styles, dlatent_size])
    dlatents_in = tf.cast(dlatents_in, dtype)
    with tf.variable_scope('lod', reuse = tf.AUTO_REUSE):
        lod_in = tf.cast(tf.get_variable('lod', initializer=np.float32(0), trainable=False), dtype)

    # Noise inputs.
    noise_inputs = []
    if use_noise:
      with tf.variable_scope('Noise', reuse=tf.AUTO_REUSE):
        for layer_idx in range(num_layers):
            res = layer_idx // 2 + 2
            shape = [1, use_noise, 2**res, 2**res]
            noise_inputs.append(tf.get_variable('noise%d' % layer_idx, shape=shape, initializer=tf.initializers.random_normal(), trainable=False))

    # Things to do at the end of each layer.
    def layer_epilogue(x, layer_idx):
        if use_noise:
            x = apply_noise(x, noise_inputs[layer_idx], randomize_noise=randomize_noise)
        x = apply_bias(x)
        x = act(x)
        if use_pixel_norm:
            x = pixel_norm(x)
        if use_instance_norm:
            x = instance_norm(x)
        if use_styles:
            x = style_mod(x, dlatents_in[:, layer_idx], use_wscale=use_wscale)
        return x

    # Early layers.
    with tf.variable_scope('4x4', reuse = tf.AUTO_REUSE):
        if const_input_layer:
            with tf.variable_scope('Const'):
                x = tf.get_variable('const', shape=[1, nf(1), 4, 4], initializer=tf.initializers.ones())
                x = layer_epilogue(tf.tile(tf.cast(x, dtype), [tf.shape(dlatents_in)[0], 1, 1, 1]), 0)
        else:
            with tf.variable_scope('Dense'):
                x = dense(dlatents_in[:, 0], fmaps=nf(1)*16, gain=gain/4, use_wscale=use_wscale) # tweak gain to match the official implementation of Progressing GAN
                x = layer_epilogue(tf.reshape(x, [-1, nf(1), 4, 4]), 0)
        with tf.variable_scope('Conv', reuse = tf.AUTO_REUSE):
            x = layer_epilogue(conv2d(x, fmaps=nf(1), kernel=3, gain=gain, use_wscale=use_wscale), 1)

    # Building blocks for remaining layers.
    def block(res, x): # res = 3..resolution_log2
        with tf.variable_scope('%dx%d' % (2**res, 2**res), reuse = tf.AUTO_REUSE):
            with tf.variable_scope('Conv0_up'):
                x = layer_epilogue(blur(upscale2d_conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale, fused_scale=fused_scale)), res*2-4)
            with tf.variable_scope('Conv1'):
                x = layer_epilogue(conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale), res*2-3)
            return x
    def torgb(res, x): # res = 2..resolution_log2
        lod = resolution_log2 - res
        with tf.variable_scope('ToRGB_lod%d' % lod, reuse = tf.AUTO_REUSE):
            return apply_bias(conv2d(x, fmaps=num_channels, kernel=1, gain=1, use_wscale=use_wscale))

    # Fixed structure: simple and efficient, but does not support progressive growing.
    if structure == 'fixed':
        for res in range(3, resolution_log2 + 1):
            x = block(res, x)
        images_out = torgb(resolution_log2, x)

    # Linear structure: simple but inefficient.
    if structure == 'linear':
        images_out = torgb(2, x)
        for res in range(3, resolution_log2 + 1):
            lod = resolution_log2 - res
            x = block(res, x)
            img = torgb(res, x)
            images_out = upscale2d(images_out)
            with tf.variable_scope('Grow_lod%d' % lod, reuse = tf.AUTO_REUSE):
                images_out = tflib.lerp_clip(img, images_out, lod_in - lod)

    # Recursive structure: complex but efficient.
    if structure == 'recursive':
        def cset(cur_lambda, new_cond, new_lambda):
            return lambda: tf.cond(new_cond, new_lambda, cur_lambda)
        def grow(x, res, lod):
            y = block(res, x)
            img = lambda: upscale2d(torgb(res, y), 2**lod)
            img = cset(img, (lod_in > lod), lambda: upscale2d(tflib.lerp(torgb(res, y), upscale2d(torgb(res - 1, x)), lod_in - lod), 2**lod))
            if lod > 0: img = cset(img, (lod_in < lod), lambda: grow(y, res + 1, lod - 1))
            return img()
        images_out = grow(x, 3, resolution_log2 - 3)

    assert images_out.dtype == tf.as_dtype(dtype)
    return tf.identity(images_out, name='images_out')

In [0]:
def G1(
    dlatents_in,
    dlatent_size = 512,
    num_channels = 3,
    resolution = 256,
    fmap_base = 8192,
    fmap_decay = 1.0,
    fmap_max = 512,
    use_style = True,
    const_input_layer = True,
    use_noise = True,
    randomize_noise = True,
    nonlinearity = 'lrelu',
    use_wscale = True,
    use_pixel_norm = False,
    use_instance_norm = True,
    dtype = 'float32',
    fused_scale = True,
    blur_filter = [1,2,1],
    structure = 'linear',
    is_template_graph = False,
    force_clean_graph = False,
    **_kwargs):
    
  
    resolution_log2 = int(np.log2(resolution))
    assert resolution == 2**resolution_log2 and resolution >= 4
    def nf(stage): return min(int(fmap_base / (2.0 ** (stage * fmap_decay))), fmap_max)
    def blur(x): return blur2d(x, blur_filter) if blur_filter else x
    if is_template_graph: force_clean_graph = True
    if force_clean_graph: randomize_noise = False
    if structure == 'auto': structure = 'linear' if force_clean_graph else 'recursive'
    act, gain = {'relu': (tf.nn.relu, np.sqrt(2)), 'lrelu': (leaky_relu, np.sqrt(2))}[nonlinearity]
    num_layers = resolution_log2 * 2 - 2
    num_styles = num_layers if use_style else 1
    images_out = None


    #dlatents_in = tf.cast(dlatents_in, dtype)
    with tf.variable_scope('lod', reuse=tf.AUTO_REUSE):
        lod_in = tf.cast(tf.get_variable('lod', initializer=np.float32(0), trainable=False), dtype)

    # Noise inputs.
    noise_inputs = []
    if use_noise:
      with tf.variable_scope('Noise', reuse=tf.AUTO_REUSE):
        for layer_idx in range(num_layers):
            res = layer_idx // 2 + 2
            shape = [1, use_noise, 2**res, 2**res]
            noise_inputs.append(tf.get_variable('noise%d' % layer_idx, shape=shape, initializer=tf.initializers.random_normal(), trainable=False))

    a1 = []
    a11 = []
    a2 = []
    a3 = []
    a4 = []
    a5 = []
    a6 = []
    a71 = []
    a72= []
    a8 = []
    a10 = []

    a1.append(dlatents_in)

    # Things to do at the end of each layer.
    def layer_epilogue(x, a2, a3, a4, a5, a6, a11, layer_idx):
        if use_noise:
            x = apply_noise(x, noise_inputs[layer_idx], randomize_noise=randomize_noise)
            a2.append(x)
        x = apply_bias(x)
        a3.append(x)
        x = act(x)
        a4.append(x)
        if use_pixel_norm:
            x = pixel_norm(x)
        if use_instance_norm:
            x = instance_norm(x)
            a5.append(x)
        if use_style:
            x = style_mod(x, dlatents_in[:, layer_idx], use_wscale = use_wscale)
            a6.append(x)
        return x

    def block(res, a1, a2, a3, a4, a5, a6, a11, x): # res = 3..resolution_log2
        with tf.variable_scope('%dx%d' % (2**res, 2**res), reuse=tf.AUTO_REUSE):
            with tf.variable_scope('Conv0_up'):
                x = blur(upscale2d_conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale, fused_scale=fused_scale))
                a1.append(x)
                x = layer_epilogue(x, a2, a3, a4, a5, a6, a11, res*2-4)
            with tf.variable_scope('Conv1'):
                x = conv2d(x, fmaps=nf(res-1), kernel=3, gain=gain, use_wscale=use_wscale)
                a1.append(x)
                x = layer_epilogue(x, a2, a3, a4, a5, a6, a11, res*2-3)
            return x

    def torgb(res, a71, a72,  x): # res = 2..resolution_log2
        lod = resolution_log2 - res
        with tf.variable_scope('ToRGB_lod%d' % (lod) , reuse = tf.AUTO_REUSE):
            x = conv2d(x, fmaps=num_channels, kernel=1, gain=1, use_wscale=use_wscale)
            a71.append(x)
            x = apply_bias(x)
            a72.append(x)
            return x

    # Early layers.
    with tf.variable_scope('4x4', reuse=tf.AUTO_REUSE):
        if const_input_layer:
            with tf.variable_scope('Const'):
                x = tf.get_variable('const', shape=[1, nf(1), 4, 4], initializer=tf.initializers.ones())
                a1.append(x)
                x = layer_epilogue(tf.tile(tf.cast(x, dtype), [tf.shape(dlatents_in)[0], 1, 1, 1]), a2, a3, a4, a5, a6, a11, 0)
        else:
            with tf.variable_scope('Dense'):
                x = dense(dlatents_in[:, 0], fmaps=nf(1)*16, gain=gain/4, use_wscale=use_wscale) # tweak gain to match the official implementation of Progressing GAN
                x = layer_epilogue(tf.reshape(x, [-1, nf(1), 4, 4]), a2, a3, a4, a5, a6, a11, 0)
        with tf.variable_scope('Conv'):
            x = conv2d(x, fmaps=nf(1), kernel=3, gain=gain, use_wscale=use_wscale)
            x = layer_epilogue(x, a2, a3, a4, a5, a6, a11, 1)

    if structure == 'fixed':
        for res in range(3, resolution_log2 + 1):
            x = block(res, a1, a2, a3, a4, a5, a6, x)
        images_out = torgb(resolution_log2, a7, x)

    if structure == 'linear':
        print("Structure followed is Linear")
        images_out = torgb(2, a71, a72, x)
        a10.append(images_out)
        for res in range(3, resolution_log2 + 1):
            lod = resolution_log2 - res
            x = block(res, a1, a2, a3, a4, a5, a6, a11, x)
            img = torgb(res, a71, a72, x)
            images_out = upscale2d(images_out)
            with tf.variable_scope('Grow_lod%d' % lod, reuse = tf.AUTO_REUSE):
                images_out = tflib.lerp_clip(img, images_out, lod_in - lod)
                a10.append(images_out)

    a8.append(a1)
    a8.append(a2)
    a8.append(a3)
    a8.append(a4)
    a8.append(a5)
    a8.append(a6)
    a8.append(a71)
    a8.append(a72)
    a8.append(a10)

    return a8, tf.identity(images_out, name='images_out')

In [0]:
cd /content/

/content


In [0]:
import pickle
with open("noise.txt", "rb") as fp:   
   noise = pickle.load(fp)

In [0]:
import pickle
with open("const.txt", "rb") as fp:   
   const = pickle.load(fp)

In [0]:

with open("input1.txt", "rb") as fp:   
   input1 = pickle.load(fp)

In [0]:
with open("initial_conv_weights1.txt", "rb") as fp:   
   initial_conv_weights1 = pickle.load(fp)

In [0]:
with open("initial_conv_weights2.txt", "rb") as fp:   
   initial_conv_weights2 = pickle.load(fp)

In [0]:
with open("initial_conv_weights3.txt", "rb") as fp:   
   initial_conv_weights3 = pickle.load(fp)

In [0]:
with open("initial_conv_weights4.txt", "rb") as fp:   
   initial_conv_weights4 = pickle.load(fp)

In [0]:
with open("initial_conv_weights5.txt", "rb") as fp:   
   initial_conv_weights5 = pickle.load(fp)

In [0]:
with open("initial_conv_weights6.txt", "rb") as fp:   
   initial_conv_weights6 = pickle.load(fp)

In [0]:
with open("initial_conv_weights7.txt", "rb") as fp:   
   initial_conv_weights7 = pickle.load(fp)

In [0]:
cd /content/stylegan/

/content/stylegan


In [0]:
inputx = input1[0]
inputx = tf.convert_to_tensor(inputx, dtype = tf.float32)

In [0]:
list1, out1 = G_synthesis12(inputx, num_channels = 3, resolution= 256, randomize_noise = False, use_wscale = False, fused_scale=False, structure='linear')

In [0]:
print(out1.shape)

(1, 3, 256, 256)


In [0]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [0]:
o1 = G_synthesis(inputx, num_channels = 3, resolution= 256, randomize_noise = False, use_wscale = False)

In [0]:
print(o1.shape)

(1, 3, 256, 256)


In [0]:
list1, output1 = G1(input, randomize_noise = False, use_wscale = False)

Structure followed is Linear


In [0]:
print(output1.shape)

(1, 3, 256, 256)


In [0]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [0]:
b1 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Noise')
b2 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='4x4/Const')
b3 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='4x4/Conv')
b4 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod6')
b51 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='8x8/Conv0_up')
b61 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='8x8/Conv1')
b71 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod5')
b52 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='16x16/Conv0_up')
b62 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='16x16/Conv1')
b72= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod4')
b53 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='32x32/Conv0_up')
b63 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='32x32/Conv1')
b73 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod3')
b54 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='64x64/Conv0_up')
b64 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='64x64/Conv1')
b74= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod2')
b55 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='128x128/Conv0_up')
b65 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='128x128/Conv1')
b75 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod1')
b56 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='256x256/Conv0_up')
b66 = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='256x256/Conv1')
b76= tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ToRGB_lod0')

In [0]:
print(len(b1))
print(len(b2))
print(len(b3))
print(len(b4))
print(len(b51))
print(len(b61))
print(len(b71))
print(len(b52))
print(len(b62))
print(len(b72))
print(len(b53))
print(len(b63))
print(len(b73))
print(len(b54))
print(len(b64))
print(len(b74))
print(len(b55))
print(len(b65))
print(len(b75))
print(len(b56))
print(len(b66))
print(len(b76))

14
5
5
2
5
5
2
5
5
2
5
5
2
5
5
2
5
5
2
5
5
2


In [0]:
l = 0
for i in b1:
    print("%d Noise is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(noise[l])))
    l= l + 1

1 Noise is:
2 Noise is:
3 Noise is:
4 Noise is:
5 Noise is:
6 Noise is:
7 Noise is:
8 Noise is:
9 Noise is:
10 Noise is:
11 Noise is:
12 Noise is:
13 Noise is:
14 Noise is:


In [0]:
l = 0
for i in b2:
    print("%d Noise is:" %(l+1))
    if(l==0):
        print(sess.run(i[0][0][0][0]))
    sess.run(i.assign(tf.convert_to_tensor(const[l])))
    if(l==0):
        print(sess.run(i[0][0][0][0]))
        print(const[l][0][0][0][0])
    print(" ")
    l = l + 1

1 Noise is:
1.0
-1.84618
-1.84618
 
2 Noise is:
 
3 Noise is:
 
4 Noise is:
 
5 Noise is:
 


In [0]:
l = 0
for i in b3:
    print("%d Initial Convolution Weights is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights1[l])))
    print(" ")
    l= l + 1

for i in b4:
    print("%d Initial Convolution Weights is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights1[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights is:
 
2 Initial Convolution Weights is:
 
3 Initial Convolution Weights is:
 
4 Initial Convolution Weights is:
 
5 Initial Convolution Weights is:
 
6 Initial Convolution Weights is:
 
7 Initial Convolution Weights is:
 


In [0]:
l = 0
for i in b51:
    print("%d Initial Convolution Weights 2 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights2[l])))
    print(" ")
    l= l + 1

for i in b61:
    print("%d Initial Convolution Weights 2 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights2[l])))
    print(" ")
    l= l + 1

for i in b71:
    print("%d Initial Convolution Weights 2 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights2[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights 2 is:
 
2 Initial Convolution Weights 2 is:
 
3 Initial Convolution Weights 2 is:
 
4 Initial Convolution Weights 2 is:
 
5 Initial Convolution Weights 2 is:
 
6 Initial Convolution Weights 2 is:
 
7 Initial Convolution Weights 2 is:
 
8 Initial Convolution Weights 2 is:
 
9 Initial Convolution Weights 2 is:
 
10 Initial Convolution Weights 2 is:
 
11 Initial Convolution Weights 2 is:
 
12 Initial Convolution Weights 2 is:
 


In [0]:
l = 0
for i in b52:
    print("%d Initial Convolution Weights 3 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights3[l])))
    print(" ")
    l= l + 1

for i in b62:
    print("%d Initial Convolution Weights 3 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights3[l])))
    print(" ")
    l= l + 1

for i in b72:
    print("%d Initial Convolution Weights 3 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights3[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights 3 is:
 
2 Initial Convolution Weights 3 is:
 
3 Initial Convolution Weights 3 is:
 
4 Initial Convolution Weights 3 is:
 
5 Initial Convolution Weights 3 is:
 
6 Initial Convolution Weights 3 is:
 
7 Initial Convolution Weights 3 is:
 
8 Initial Convolution Weights 3 is:
 
9 Initial Convolution Weights 3 is:
 
10 Initial Convolution Weights 3 is:
 
11 Initial Convolution Weights 3 is:
 
12 Initial Convolution Weights 3 is:
 


In [0]:
l = 0
for i in b53:
    print("%d Initial Convolution Weights 4 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights4[l])))
    print(" ")
    l= l + 1

for i in b63:
    print("%d Initial Convolution Weights 4 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights4[l])))
    print(" ")
    l= l + 1

for i in b73:
    print("%d Initial Convolution Weights 4 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights4[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights 4 is:
 
2 Initial Convolution Weights 4 is:
 
3 Initial Convolution Weights 4 is:
 
4 Initial Convolution Weights 4 is:
 
5 Initial Convolution Weights 4 is:
 
6 Initial Convolution Weights 4 is:
 
7 Initial Convolution Weights 4 is:
 
8 Initial Convolution Weights 4 is:
 
9 Initial Convolution Weights 4 is:
 
10 Initial Convolution Weights 4 is:
 
11 Initial Convolution Weights 4 is:
 
12 Initial Convolution Weights 4 is:
 


In [0]:
l = 0
for i in b54:
    print("%d Initial Convolution Weights 5 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights5[l])))
    print(" ")
    l= l + 1

for i in b64:
    print("%d Initial Convolution Weights 5 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights5[l])))
    print(" ")
    l= l + 1

for i in b74:
    print("%d Initial Convolution Weights 5 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights5[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights 5 is:
 
2 Initial Convolution Weights 5 is:
 
3 Initial Convolution Weights 5 is:
 
4 Initial Convolution Weights 5 is:
 
5 Initial Convolution Weights 5 is:
 
6 Initial Convolution Weights 5 is:
 
7 Initial Convolution Weights 5 is:
 
8 Initial Convolution Weights 5 is:
 
9 Initial Convolution Weights 5 is:
 
10 Initial Convolution Weights 5 is:
 
11 Initial Convolution Weights 5 is:
 
12 Initial Convolution Weights 5 is:
 


In [0]:
l = 0
for i in b55:
    print("%d Initial Convolution Weights 6 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights6[l])))
    print(" ")
    l= l + 1

for i in b65:
    print("%d Initial Convolution Weights 6 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights6[l])))
    print(" ")
    l= l + 1

for i in b75:
    print("%d Initial Convolution Weights 6 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights6[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights 6 is:
 
2 Initial Convolution Weights 6 is:
 
3 Initial Convolution Weights 6 is:
 
4 Initial Convolution Weights 6 is:
 
5 Initial Convolution Weights 6 is:
 
6 Initial Convolution Weights 6 is:
 
7 Initial Convolution Weights 6 is:
 
8 Initial Convolution Weights 6 is:
 
9 Initial Convolution Weights 6 is:
 
10 Initial Convolution Weights 6 is:
 
11 Initial Convolution Weights 6 is:
 
12 Initial Convolution Weights 6 is:
 


In [0]:
l = 0
for i in b56:
    print("%d Initial Convolution Weights 7 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights7[l])))
    print(" ")
    l= l + 1

for i in b66:
    print("%d Initial Convolution Weights 7 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights7[l])))
    print(" ")
    l= l + 1

for i in b76:
    print("%d Initial Convolution Weights 7 is:" %(l+1))
    sess.run(i.assign(tf.convert_to_tensor(initial_conv_weights7[l])))
    print(" ")
    l= l + 1

1 Initial Convolution Weights 7 is:
 
2 Initial Convolution Weights 7 is:
 
3 Initial Convolution Weights 7 is:
 
4 Initial Convolution Weights 7 is:
 
5 Initial Convolution Weights 7 is:
 
6 Initial Convolution Weights 7 is:
 
7 Initial Convolution Weights 7 is:
 
8 Initial Convolution Weights 7 is:
 
9 Initial Convolution Weights 7 is:
 
10 Initial Convolution Weights 7 is:
 
11 Initial Convolution Weights 7 is:
 
12 Initial Convolution Weights 7 is:
 


In [0]:
lis2, out2 = G_synthesis12(inputx, num_channels = 3, resolution= 256, randomize_noise = False, use_wscale = False, fused_scale=False, structure='linear')

In [0]:
print(sess.run(out2[0][0][0][0]))
print(sess.run(out2[0][0][0][1]))
print(sess.run(out2[0][0][0][2]))
print(sess.run(out2[0][0][0][3]))

11.214869
-12.609013
-23.258228
-49.759335


In [0]:
l = 0
for i in lis2[0]:
  if(l>0):
    print("%d Output is:" %(l))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print()
  else:
    print("1St Input is:")
    print(sess.run(i[0][0][0]))
    print(sess.run(i[0][0][1]))
    print(sess.run(i[0][0][2]))
    print()
  l = l + 1

1St Input is:
0.83128
0.2854586
0.9196952

1 Output is:
-1.84618
6.3599644
0.12951864

2 Output is:
20270.738
-104380.0
-258549.94

3 Output is:
-65122.805
-94974.875
-38889.06

4 Output is:
-7454.548
-33308.004
-38218.707

5 Output is:
89440.445
60078.76
51687.977

6 Output is:
27369.43
15288.547
22480.377

7 Output is:
-8352.848
-9601.129
-8677.818

8 Output is:
-87.29541
7417.2812
6112.753



In [0]:
l = 0
for i in lis2[1]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-86534.234
-139699.39
-147677.02
-152247.47

2 Output is:
-81667.58
-115441.414
-106167.01
-106196.555

3 Output is:
139967.61
136911.22
79867.53
86454.64

4 Output is:
196072.95
297424.25
302385.38
295122.44

5 Output is:
933.8535
9375.004
13146.991
19079.512

6 Output is:
-9602.999
-4809.956
-8587.126
-20599.746



In [0]:
l = 0
for i in lis2[2]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-47362.38
-77966.36
-93147.164
-101218.66

2 Output is:
-58360.664
-88165.27
-92632.87
-89882.06

3 Output is:
84949.38
100646.77
74192.13
61205.906

4 Output is:
151131.81
236022.22
247845.23
208118.33

5 Output is:
2486.7075
6073.914
7059.1035
4319.425

6 Output is:
-882.70386
660.4984
-1894.5182
-8439.295



In [0]:
l = 0
for i in lis2[3]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-2.0104156
6.4205313
0.12872617
-7.7757373

2 Output is:
20269.348
-104383.76
-258550.44
-126136.34

3 Output is:
-47362.94
-77965.73
-93149.016
-101218.914

4 Output is:
-65122.85
-94974.91
-38889.05
18012.846

5 Output is:
-58360.707
-88165.01
-92632.46
-89882.445

6 Output is:
-7455.2754
-33308.39
-38218.047
-38343.99

7 Output is:
84949.734
100647.016
74197.164
61209.61

8 Output is:
89439.76
60077.12
51687.305
40187.387

9 Output is:
151131.06
236021.9
247845.4
208116.77

10 Output is:
27369.521
15288.609
22480.375
30243.58

11 Output is:
2486.7183
6073.941
7059.193
4319.41

12 Output is:
-8352.838
-9601.122
-8677.796
-10648.09

13 Output is:
-883.53485
657.83673
-1892.0635
-8441.823

14 Output is:
-87.298645
7417.4097
6112.742
5586.7993



In [0]:
l = 0
for i in lis2[4]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-0.77507746
7.6558695
1.3640642
-6.540399

2 Output is:
20269.697
-104383.41
-258550.1
-126135.99

3 Output is:
-47360.027
-77962.81
-93146.1
-101216.0

4 Output is:
-65122.117
-94974.17
-38888.316
18013.578

5 Output is:
-58355.426
-88159.73
-92627.18
-89877.164

6 Output is:
-7452.5596
-33305.676
-38215.332
-38341.273

7 Output is:
84959.84
100657.125
74207.27
61219.715

8 Output is:
89438.0
60075.363
51685.547
40185.63

9 Output is:
151126.75
236017.6
247841.1
208112.45

10 Output is:
27365.73
15284.818
22476.584
30239.79

11 Output is:
2486.5928
6073.8154
7059.0674
4319.2847

12 Output is:
-8352.869
-9601.153
-8677.827
-10648.121

13 Output is:
-862.9659
678.4057
-1871.4945
-8421.254

14 Output is:
-86.26122
7418.4473
6113.78
5587.837



In [0]:
l = 0
for i in lis2[5]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-0.1550155
7.6558695
1.3640642
-1.3080798

2 Output is:
20269.697
-20876.682
-51710.02
-25227.2

3 Output is:
-9472.006
-15592.5625
-18629.22
-20243.201

4 Output is:
-13024.424
-18994.834
-7777.6636
18013.578

5 Output is:
-11671.085
-17631.945
-18525.436
-17975.434

6 Output is:
-1490.512
-6661.1353
-7643.0664
-7668.255

7 Output is:
84959.84
100657.125
74207.27
61219.715

8 Output is:
89438.0
60075.363
51685.547
40185.63

9 Output is:
151126.75
236017.6
247841.1
208112.45

10 Output is:
27365.73
15284.818
22476.584
30239.79

11 Output is:
2486.5928
6073.8154
7059.0674
4319.2847

12 Output is:
-1670.5739
-1920.2307
-1735.5654
-2129.6243

13 Output is:
-172.59319
678.4057
-374.29892
-1684.2509

14 Output is:
-17.252245
7418.4473
6113.78
5587.837



In [0]:
l = 0
for i in lis2[6]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-1.0466243
1.964768
-0.46096146
-1.4911743

2 Output is:
1.3550243
-0.5365803
-1.954068
-0.7365848

3 Output is:
1.3220385
0.24045923
-0.29615647
-0.58136714

4 Output is:
-1.67347
-1.7337677
-1.6204807
-1.3600038

5 Output is:
1.0940474
0.088608876
-0.06209915
0.030671543

6 Output is:
-1.0768757
-1.1827168
-1.2028167
-1.2033323

7 Output is:
0.21005447
0.4979605
0.0128400875
-0.22536653

8 Output is:
0.8832909
0.43437082
0.3061004
0.13028023

9 Output is:
0.88094425
1.5024749
1.589041
1.2981666

10 Output is:
0.8407654
0.35147288
0.64274865
0.9571685

11 Output is:
0.0033882388
0.3352857
0.42644334
0.17295279

12 Output is:
-0.36160192
-0.46604714
-0.38879147
-0.553648

13 Output is:
-1.2150289
-1.1634358
-1.2272575
-1.3066751

14 Output is:
-0.032780968
1.1353562
0.93039495
0.8477701



In [0]:
l = 0
for i in lis2[7]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
7.758049
-216.90973
-35.93588
40.924126

2 Output is:
-136.65549
-65.117355
-11.50975
-57.553444

3 Output is:
-131.7185
-74.15669
-45.59793
-30.41898

4 Output is:
419.69623
438.08603
403.53537
324.0942

5 Output is:
37.34501
12.958047
9.302616
11.552774

6 Output is:
-75.569115
-79.91198
-80.73672
-80.75787

7 Output is:
-93.56799
-147.77458
-56.436707
-11.587456

8 Output is:
99.10971
83.77889
79.39841
73.39407

9 Output is:
329.2097
518.44196
544.79803
456.2379

10 Output is:
-141.64433
-117.61006
-131.91766
-147.3621

11 Output is:
-14.405905
-101.68496
-125.65668
-58.996284

12 Output is:
19.815739
8.009159
16.74221
-1.8933144

13 Output is:
62.318832
63.591736
62.017124
60.057728

14 Output is:
-19.65595
57.700832
44.127815
38.656204



In [0]:
list2, output2 = G1(input, use_style = True, use_noise = False, randomize_noise=False, use_wscale = False, structure = 'linear')

NameError: ignored

In [0]:
o2 = G_synthesis(inputx, num_channels = 3, resolution= 256, randomize_noise = False, use_wscale = False)

In [0]:
print(sess.run(o2[0][0][0][0]))
print(sess.run(o2[0][0][0][1]))
print(sess.run(o2[0][0][0][2]))

-59.79628
-103.082016
-116.63952


In [0]:
print(sess.run(output2[0][0][0][0]))
print(sess.run(output2[0][0][0][1]))
print(sess.run(output2[0][0][0][2]))

0.85311663
0.9419849
1.038805


In [0]:
print(input1[0][0][0][0])

0.8312799770878936


In [0]:
l = 0
for i in list2[0]:
  if(l>0):
    print("%d Output is:" %(l))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print()
  else:
    print("1St Input is:")
    print(sess.run(i[0][0][0]))
    print(sess.run(i[0][0][1]))
    print(sess.run(i[0][0][2]))
    print()
  l = l + 1

1St Input is:
0.83128
0.2854586
0.9196952

1 Output is:
-1.84618
6.3599644
0.12951864

2 Output is:
-12010.066
-31704.414
-41272.45

3 Output is:
-15597.707
-40521.11
6768.142



In [0]:
l = 0
for i in list2[1]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-2.0104156
6.4205313
0.12872617
-7.7757373

2 Output is:
20269.348
-104383.76
-258550.44
-126136.34

3 Output is:
-12010.63
-31703.781
-41274.305
-35035.707

4 Output is:
-15597.755
-40521.14
6768.1494
52193.504



In [0]:
l = 0
for i in list2[2]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-0.77507746
7.6558695
1.3640642
-6.540399

2 Output is:
20269.697
-104383.41
-258550.1
-126135.99

3 Output is:
-12007.716
-31700.867
-41271.39
-35032.793

4 Output is:
-15597.022
-40520.406
6768.8823
52194.24



In [0]:
l = 0
for i in list2[3]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-0.1550155
7.6558695
1.3640642
-1.3080798

2 Output is:
20269.697
-20876.682
-51710.02
-25227.2

3 Output is:
-2401.5432
-6340.1733
-8254.278
-7006.5586

4 Output is:
-3119.4045
-8104.0815
6768.8823
52194.24



In [0]:
l = 0
for i in list2[4]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-1.0466243
1.964768
-0.46096146
-1.4911743

2 Output is:
1.3550243
-0.5365803
-1.954068
-0.7365848

3 Output is:
2.4080899
1.7835858
1.4800879
1.6779248

4 Output is:
-1.8152492
-1.8745228
-1.6976659
-1.1575049



In [0]:
l = 0
for i in list2[5]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
7.758049
-216.90973
-35.93588
40.924126

2 Output is:
-136.65549
-65.117355
-11.50975
-57.553444

3 Output is:
-189.51831
-156.2821
-140.12991
-150.65881

4 Output is:
462.93655
481.014
427.0756
262.33536



In [0]:
l = 0
for i in list2[6]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-8141.2607
-2891.1162
-1466.6011
-1448.821

2 Output is:
907.33386
3396.8694
3403.1763
3161.273



In [0]:
l = 0
for i in list2[7]:
    print("%d Output is:" %(l+1))
    print(sess.run(i[0][0][0][0]))
    print(sess.run(i[0][0][0][1]))
    print(sess.run(i[0][0][0][2]))
    print(sess.run(i[0][0][0][3]))
    print()
    l = l + 1

1 Output is:
-8141.2607
-2891.1162
-1466.6011
-1448.821

2 Output is:
907.33386
3396.8694
3403.1763
3161.273

